In [20]:
import numpy as np

In [21]:
img = np.arange(9).reshape((3, 3)) + 1
img

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [26]:
def box_sum(img: np.ndarray, k: int) -> np.ndarray:
    h, w = img.shape
    size_y = h - k + 1
    size_x = w - k + 1
    cum_cols = np.cumsum(img, axis=0)
    ii = np.cumsum(cum_cols, axis=1)

    print(cum_cols)

    return ii

In [27]:
box_sum(img, 2)

[[ 1  2  3]
 [ 5  7  9]
 [12 15 18]]


array([[ 1,  3,  6],
       [ 5, 12, 21],
       [12, 27, 45]])